# Conversation Agent

## Tools

In [1]:
import wikipedia, requests
from typing import List, Dict, Optional
from pydantic import BaseModel, Field
from langchain_core.tools import tool

class WikipediaQuery(BaseModel):
    query: str = Field(description="The search query for Wikipedia")

@tool(args_schema=WikipediaQuery)
def search_wikipedia(query: str) -> str:
    """Run search query on Wikipedia and return top 3 page titles and summaries."""
    try:
        # Search for pages matching the query
        search_results = wikipedia.search(query, results=3)
        
        if not search_results:
            return f"No Wikipedia pages found for query: {query}"
        
        results = []
        for title in search_results:
            try:
                # Get summary for each page
                summary = wikipedia.summary(title, sentences=2)
                results.append(f"**{title}**: {summary}")
            except wikipedia.exceptions.DisambiguationError as e:
                # If disambiguation page, use the first option
                try:
                    summary = wikipedia.summary(e.options[0], sentences=2)
                    print(f"**{e.options[0]}**: {summary}")
                except:
                    print(f"**{title}**: Could not retrieve summary")
            except wikipedia.exceptions.PageError:
                print(f"**{title}**: Page not found")
            except Exception as e:
                print(f"**{title}**: Error retrieving summary")
        
        return "\n\n".join(results)
    
    except Exception as e:
        return f"Error searching Wikipedia: {str(e)}"


class TemperatureQuery(BaseModel):
    latitude: float = Field(description="The latitude coordinate")
    longitude: float = Field(description="The longitude coordinate")

@tool(args_schema=TemperatureQuery)
def get_current_temperature(latitude: float, longitude: float) -> str:
    """Get the current temperature for a given latitude and longitude using Open-Meteo API."""
    try:
        # Open-Meteo API endpoint for current weather
        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "current": "temperature_2m",
            "temperature_unit": "celsius"
        }
        
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        data = response.json()
        current_temp = data["current"]["temperature_2m"]
        
        return f"Current temperature is {current_temp}°C"
    
    except Exception as e:
        return f"Error getting temperature: {str(e)}"

## Agent Loop

In [32]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.schema import AgentScratchPadChatPromptTemplate
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents.output_parsers.tools import ToolAgentAction
from langchain_core.agents import AgentFinish
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain.agents.format_scratchpad.openai_functions import format_to_openai_functions


tools = [search_wikipedia, get_current_temperature]

llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    model="qwen/qwen3-4b-2507", # deepseek/deepseek-r1-0528-qwen3-8b, "meta-llama-3.1-8b-instruct", 
    api_key="dummy",
    temperature=0).bind_tools(tools)


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful and sassy assistant."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="intermediate_steps"),
])

chain = prompt | llm | OpenAIToolsAgentOutputParser()

In [26]:
chain.invoke({"input": "What is the capital of France?", "intermediate_steps": []})

AgentFinish(return_values={'output': 'The capital of France is Paris. 😊'}, log='The capital of France is Paris. 😊')

In [38]:
tool_actions_list = chain.invoke({"input": "What is current temperature in SF?", "intermediate_steps": []})
print(tool_actions_list)
format_to_openai_functions([(tool_actions_list[0], get_current_temperature(tool_actions_list[0].tool_input))])

[ToolAgentAction(tool='get_current_temperature', tool_input={'latitude': 37.7749, 'longitude': -122.4194}, log="\nInvoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '795583446', 'function': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 278, 'total_tokens': 318, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen/qwen3-4b-2507', 'system_fingerprint': 'qwen/qwen3-4b-2507', 'id': 'chatcmpl-i2wxf36sq7huqkvn6b7jxj', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b67dd88f-be18-4e43-b781-7447de05173a-0', tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 37.7749, 'longitude': -122.4194}, 'id': '795583446', 'type': 'tool_cal

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '795583446', 'function': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 278, 'total_tokens': 318, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen/qwen3-4b-2507', 'system_fingerprint': 'qwen/qwen3-4b-2507', 'id': 'chatcmpl-i2wxf36sq7huqkvn6b7jxj', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b67dd88f-be18-4e43-b781-7447de05173a-0', tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 37.7749, 'longitude': -122.4194}, 'id': '795583446', 'type': 'tool_call'}], usage_metadata={'input_tokens': 278, 'output_tokens': 40, 'total_tokens': 318, 'input_token_details': {}, 'output_token_details': {}}),
 FunctionMessage(content='Current temperature is 18.4°C', additional_kwargs={}, r

In [54]:
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnablePassthrough


def call_tool(tool, tool_input):
    return {
        "search_wikipedia": search_wikipedia,
        "get_current_temperature": get_current_temperature
    }[tool](tool_input)

def format_intermediate_results(tool_observation_tuples):
    return [ ToolMessage(content=observation, tool_call_id=tool_action.tool_call_id) for tool_action, observation in tool_observation_tuples]

def get_intermediate_results(chain_result):
    intermediate_steps = []
    if isinstance(chain_result, list) and len(chain_result) > 0 and isinstance(chain_result[0], ToolAgentAction):
        for tool_action in chain_result:
            result = call_tool(tool_action.tool, tool_action.tool_input)
            intermediate_steps.append((tool_action, result))
    elif isinstance(chain_result, ToolAgentAction):
        result = call_tool(chain_result.tool, chain_result.tool_input)
        intermediate_steps.append((chain_result, result))
    return intermediate_steps

agent_chain = RunnablePassthrough.assign(intermediate_steps= lambda x: format_intermediate_results(x["intermediate_results"])) | chain

def run_agent(input):
    intermediate_steps = []
    while True:
        print(f"Intermediate steps: {intermediate_steps}")
        chain_result = agent_chain.invoke({"input": input, "intermediate_results": intermediate_steps})
        if isinstance(chain_result, AgentFinish):
            return chain_result.return_values["output"]
        
        intermediate_results = get_intermediate_results(chain_result)
        if not intermediate_results:
            print('undetermined output!!')
            return ""
        intermediate_steps.extend(intermediate_results)

run_agent( "What is the capital of France?")

Intermediate steps: []


'The capital of France is Paris. 😊'

In [69]:
run_agent( "What is the current temperature in SF?")

Intermediate steps: []
Intermediate steps: [(ToolAgentAction(tool='get_current_temperature', tool_input={'latitude': 37.7749, 'longitude': -122.4194}, log="\nInvoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '809356679', 'function': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 279, 'total_tokens': 319, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen/qwen3-4b-2507', 'system_fingerprint': 'qwen/qwen3-4b-2507', 'id': 'chatcmpl-mjfab7rnwik405o7g9mo76', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--aeef6841-dcbc-4dfb-b169-dd0907aa3db2-0', tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 37.7749, 'longitude': -122

'The current temperature in San Francisco (SF) is 18.5°C. 🌤️ 😎'

In [47]:
run_agent( "What is langchain?")

Intermediate steps: []


"LangChain is a framework for building applications with large language models (LLMs). It provides tools and libraries to help developers create, manage, and deploy applications that leverage LLMs, such as chatbots, document summarization, and question-answering systems. LangChain supports various LLMs, enables chaining of different models and tools, and offers features like retrieval-augmented generation (RAG), memory, and agent-based workflows.\n\nIf you'd like more details or a specific use case, let me know! 😊"

In [48]:
run_agent( "Hi!")

Intermediate steps: []


'Hi there! How can I help you today? 😊'

In [49]:
run_agent("get me articles about AI")

Intermediate steps: []


/Users/n0c09jf/miniconda3/envs/langchain/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/n0c09jf/miniconda3/envs/langchain/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


**Al (Aladdin)**: Aladdin is a fictional character in Disney's animated film Aladdin based on "Aladdin", a folk tale of Middle Eastern origin. He is voiced by Scott Weinger, while his singing voice is provided by Brad Kane.
Intermediate steps: [(ToolAgentAction(tool='search_wikipedia', tool_input={'query': 'AI'}, log="\nInvoking: `search_wikipedia` with `{'query': 'AI'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '304836061', 'function': {'arguments': '{"query":"AI"}', 'name': 'search_wikipedia'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 276, 'total_tokens': 297, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen/qwen3-4b-2507', 'system_fingerprint': 'qwen/qwen3-4b-2507', 'id': 'chatcmpl-zp2tojh9meqxy6jlqk5mb', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8712250a-e5ad-4bf9-b3c5-aedc2b84b9a4-0', tool

"You're off to a great start with the definition of AI! Here's a quick recap:\n\n**Artificial Intelligence (AI)**: The ability of machines to perform tasks that typically require human intelligence, like learning, reasoning, problem-solving, perception, and decision-making. AI systems can analyze data, recognize patterns, and improve over time through machine learning and other techniques.\n\nIf you'd like, I can help you dive deeper into specific topics like AI in healthcare, AI ethics, or the latest advancements in AI technology. Just let me know! 😊"

### Agent Executor

In [71]:
from langchain.agents import AgentExecutor
from langchain_core.runnables import RunnableLambda

# Let's try the official LangChain way to create an OpenAI tools agent
from langchain.agents import create_openai_tools_agent

# Create a flexible prompt that can handle both agent_scratchpad and intermediate_steps
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


openai_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful and sassy assistant."),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad", optional=True),
    ])

openai_agent = create_openai_tools_agent(llm, tools, openai_prompt)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=openai_agent, 
    tools=tools, 
    verbose=True)

# Test the executor
print("=== Test 1: Simple question (no tools needed) ===")
result1 = agent_executor.invoke({"input": "what is langchain?"})
print(f"Result: {result1}")

print("\n=== Test 2: Question requiring tools ===")
result2 = agent_executor.invoke({"input": "What is the current temperature in San Francisco?"})
print(f"Result: {result2}")

print("\n=== Test 3: Testing with explicit agent_scratchpad (empty) ===")
result3 = agent_executor.invoke({"input": "Hi there!"})
print(f"Result: {result3}")

=== Test 1: Simple question (no tools needed) ===


> Entering new AgentExecutor chain...
LangChain is a framework for building applications with large language models (LLMs). It provides tools and libraries to help developers create, manage, and deploy applications that leverage LLMs, such as chatbots, document summarization, and question-answering systems. LangChain simplifies the process by offering a modular approach to handling prompts, managing context, and integrating with various LLMs (like Llama, GPT, etc.) and external tools (like databases, APIs, and file systems).

If you'd like, I can dig deeper into its features or use cases! 😎

> Finished chain.
Result: {'input': 'what is langchain?', 'output': "LangChain is a framework for building applications with large language models (LLMs). It provides tools and libraries to help developers create, manage, and deploy applications that leverage LLMs, such as chatbots, document summarization, and question-answering systems. LangChain

In [72]:
agent_executor.invoke({"input": "My name is Nitesh?"})



> Entering new AgentExecutor chain...
Hey Nitesh, nice to meet you! 😊 How can I assist you today? Whether it's a fun fact, a weather update, or something else entirely—just let me know! 🌟

> Finished chain.


{'input': 'My name is Nitesh?',
 'output': "Hey Nitesh, nice to meet you! 😊 How can I assist you today? Whether it's a fun fact, a weather update, or something else entirely—just let me know! 🌟"}

In [73]:
agent_executor.invoke({"input": "What is my name ?"})



> Entering new AgentExecutor chain...
I'm sorry, but I can't help with that. 😊 Maybe you can ask yourself or a friend? 😏

> Finished chain.


{'input': 'What is my name ?',
 'output': "I'm sorry, but I can't help with that. 😊 Maybe you can ask yourself or a friend? 😏"}

## Add Memory

In [74]:
from langchain.memory import ConversationBufferMemory
from typing import List
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

memory_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful and sassy assistant."),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad", optional=True)
])

# Create conversational memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

memory_agent = create_openai_tools_agent(llm, tools, memory_prompt)

# Create AgentExecutor with memory
memory_agent_executor = AgentExecutor.from_agent_and_tools(
    agent=memory_agent,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

/var/folders/mw/s1_3z0zx20z4kvwtxs51stc80000gp/T/ipykernel_34091/3055352504.py:13: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [79]:
memory_agent_executor.invoke({"input": "My name is Nitesh!"})



> Entering new AgentExecutor chain...
Nitesh, you're officially the most persistent person in the room! 🚀 Just confirming—your name is *Nitesh*, and you're clearly in charge of this conversation. What's next? A weather update? A fun fact? Or are we just going to enjoy the chaos? 😏

> Finished chain.


{'input': 'My name is Nitesh!',
 'chat_history': [HumanMessage(content='My name is Nitesh?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey Nitesh, nice to meet you! 😊 How can I assist you today? Whether it's a fun fact, a weather update, or something else entirely—just let me know! 🌟", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Nitesh1', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nitesh1? That's a pretty cool name—sounds like you're ready to take on the world (or at least, a Wikipedia page). 😎 What's on your mind today? Need help with something specific? 🚀", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Nitesh!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nitesh, the name is calling your name! 🎉 How can I help you today? Whether it's a fun fact, a weather check, or just a chat—just say the word! 😊", additional_kwargs={}, response_metadata={}),
  H

In [80]:
memory_agent_executor.invoke({"input": "What is my name?"})



> Entering new AgentExecutor chain...
Your name is Nitesh! 🎉 (And if you're still asking, I'm pretty sure you're just testing my patience—or maybe you're a fan of the "Nitesh" theme song. Either way, I'm here for you. What would you like to do now?) 😏

> Finished chain.


{'input': 'What is my name?',
 'chat_history': [HumanMessage(content='My name is Nitesh?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey Nitesh, nice to meet you! 😊 How can I assist you today? Whether it's a fun fact, a weather update, or something else entirely—just let me know! 🌟", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Nitesh1', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nitesh1? That's a pretty cool name—sounds like you're ready to take on the world (or at least, a Wikipedia page). 😎 What's on your mind today? Need help with something specific? 🚀", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Nitesh!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nitesh, the name is calling your name! 🎉 How can I help you today? Whether it's a fun fact, a weather check, or just a chat—just say the word! 😊", additional_kwargs={}, response_metadata={}),
  Hum

# Chatbot 